In [13]:
import pandas as pd
X_train = pd.read_csv(r"C:\Users\hi\Desktop\Github Repositories\spaceship-titanic-ml-pipeline\Data\X_train.csv")
y_train = pd.read_csv(r"C:\Users\hi\Desktop\Github Repositories\spaceship-titanic-ml-pipeline\Data\y_train.csv")
X_test = pd.read_csv(r"C:\Users\hi\Desktop\Github Repositories\spaceship-titanic-ml-pipeline\Data\X_test.csv")

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

In [15]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    8693 non-null   int64  
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   int64  
 3   Cabin         8693 non-null   object 
 4   Destination   8693 non-null   object 
 5   Age           8693 non-null   float64
 6   VIP           8693 non-null   int64  
 7   RoomService   8693 non-null   float64
 8   FoodCourt     8693 non-null   float64
 9   ShoppingMall  8693 non-null   float64
 10  Spa           8693 non-null   float64
 11  VRDeck        8693 non-null   float64
dtypes: float64(6), int64(3), object(3)
memory usage: 815.1+ KB


In [16]:
y_train.drop('Unnamed: 0', axis=1, inplace=True)
X_train.drop('Unnamed: 0', axis=1, inplace=True)

In [17]:
X_test.drop('Unnamed: 0', axis=1, inplace=True)

In [18]:
#selecting numeric and categoric columns
num_col = X_train.select_dtypes('int64','float64').columns
cat_col = X_train.select_dtypes('object').columns

In [19]:
num_col

Index(['CryoSleep', 'VIP'], dtype='object')

In [20]:
preprocessing = ColumnTransformer(
    transformers=[
        ('numeric',MinMaxScaler(),num_col),
        ('categoric',OneHotEncoder(handle_unknown='ignore'),cat_col)
    ]
)

In [21]:
model = Pipeline(
    steps=[
        ('preprocessor',preprocessing),
        ('smote',SMOTE(random_state=42)),
        ('classifier',XGBClassifier(random_state=42,eval_metric='logloss'))
    ]
)

In [22]:
print(X_train.shape)
print(y_train.shape)

(8693, 11)
(8693, 1)


In [23]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric', MinMaxScaler(),
                                                  Index(['CryoSleep', 'VIP'], dtype='object')),
                                                 ('categoric',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['HomePlanet', 'Cabin', 'Destination'], dtype='object'))])),
                ('smote', SMOTE(random_state=42)),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=No...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [25]:
y_pred = model.predict(X_test)

In [26]:
test = pd.read_csv(r"C:\Users\hi\Desktop\Github Repositories\spaceship-titanic-ml-pipeline\Data\test.csv")

In [28]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [29]:
submission_df = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Transported': y_pred
})

In [33]:
submission_df['Transported'] = submission_df.Transported.astype(bool)

In [35]:
submission_df.to_csv('XGBClassifier.csv', index=False)

In [ ]:
basicCleaningPipeline = Pipeline([
    ('NameRemove', FunctionTransformer(nameRemover)),
    ('HandleNaNs_Cat', FunctionTransformer(NaNHandler_Cat)),
    ('D_typeMatch', FunctionTransformer(TypeConverter)),
])